In [11]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

  Obtaining dependency information for openmeteo-requests from https://files.pythonhosted.org/packages/c8/01/b2dae93c5af92a95c3a744b808979c32766bc4eabdef87c2ab0cc0f57c27/openmeteo_requests-1.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for openmeteo-sdk>=1.4.0 from https://files.pythonhosted.org/packages/42/e5/db8cd595ffaa4327186bf407f310100b2c3699af4767d0f69e7c2b961329/openmeteo_sdk-1.11.4-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=24.0.0 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Obtaining dependency information for requests-cache from https://files.pythonhosted.org/packages/cc/23/863f517b5d74e297e669c88093ecd92e13a06c488e90acbd5507819049ac/requests_cache-1.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for retry-requests from https://files.pythonhosted.org/packages/b1/f3/8ce908497bebbc2790ef0

In [ ]:
# I got the weather data here for the lat, long of central Berlin
# For the year 2020, 2021, 2022
#https://open-meteo.com/en/docs/historical-weather-api#start_date=2022-01-01&end_date=2022-12-31&hourly=&daily=temperature_2m_max,temperature_2m_min,temperature_2m_mean,sunrise,sunset,daylight_duration,sunshine_duration,rain_sum,snowfall_sum,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant&timezone=Europe%2FBerlin

In [27]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.531677,
	"longitude": 13.381777,
	"start_date": "2020-01-01",
	"end_date": "2020-12-31",
	"daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "rain_sum", "snowfall_sum", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
daily_sunrise = daily.Variables(3).ValuesAsNumpy()
daily_sunset = daily.Variables(4).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(5).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(6).ValuesAsNumpy()
daily_rain_sum = daily.Variables(7).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(8).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(9).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(10).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(11).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant

daily_dataframe_2020 = pd.DataFrame(data = daily_data)
print(daily_dataframe_2020)


Coordinates 52.5483283996582°N 13.407821655273438°E
Elevation 38.0 m asl
Timezone b'Europe/Berlin' b'CEST'
Timezone difference to GMT+0 7200 s
                         date  temperature_2m_max  temperature_2m_min  \
0   2019-12-31 22:00:00+00:00              4.0585             -1.0415   
1   2020-01-01 22:00:00+00:00              5.3085             -3.6915   
2   2020-01-02 22:00:00+00:00              6.3585             -1.3915   
3   2020-01-03 22:00:00+00:00              6.3085              2.1585   
4   2020-01-04 22:00:00+00:00              3.7585             -1.8415   
..                        ...                 ...                 ...   
361 2020-12-26 22:00:00+00:00              2.4585              0.8585   
362 2020-12-27 22:00:00+00:00              4.3585              1.2085   
363 2020-12-28 22:00:00+00:00              3.9585              0.1085   
364 2020-12-29 22:00:00+00:00              4.6085             -0.4415   
365 2020-12-30 22:00:00+00:00              4.0585     

In [28]:
daily_dataframe_2020.to_csv('weather_data_2020.csv')

In [30]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.531677,
	"longitude": 13.381777,
	"start_date": "2021-01-01",
	"end_date": "2021-12-31",
	"daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "rain_sum", "snowfall_sum", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
daily_sunrise = daily.Variables(3).ValuesAsNumpy()
daily_sunset = daily.Variables(4).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(5).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(6).ValuesAsNumpy()
daily_rain_sum = daily.Variables(7).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(8).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(9).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(10).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(11).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant

daily_dataframe_2021 = pd.DataFrame(data = daily_data)
print(daily_dataframe_2021)


Coordinates 52.5483283996582°N 13.407821655273438°E
Elevation 38.0 m asl
Timezone b'Europe/Berlin' b'CEST'
Timezone difference to GMT+0 7200 s
                         date  temperature_2m_max  temperature_2m_min  \
0   2020-12-31 22:00:00+00:00            2.158500             -0.7415   
1   2021-01-01 22:00:00+00:00            2.658500             -0.9915   
2   2021-01-02 22:00:00+00:00            0.808500             -0.9915   
3   2021-01-03 22:00:00+00:00            1.258500             -0.2915   
4   2021-01-04 22:00:00+00:00            2.608500              0.2085   
..                        ...                 ...                 ...   
360 2021-12-26 22:00:00+00:00           -0.441500             -7.1915   
361 2021-12-27 22:00:00+00:00            2.058500             -3.7915   
362 2021-12-28 22:00:00+00:00            3.258500             -0.0415   
363 2021-12-29 22:00:00+00:00           11.658501              3.5585   
364 2021-12-30 22:00:00+00:00           13.058500     

In [31]:
daily_dataframe_2021.to_csv('weather_data_2021.csv')

In [32]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.531677,
	"longitude": 13.381777,
	"start_date": "2022-01-01",
	"end_date": "2022-12-31",
	"daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "rain_sum", "snowfall_sum", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
daily_sunrise = daily.Variables(3).ValuesAsNumpy()
daily_sunset = daily.Variables(4).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(5).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(6).ValuesAsNumpy()
daily_rain_sum = daily.Variables(7).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(8).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(9).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(10).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(11).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant

daily_dataframe_2022 = pd.DataFrame(data = daily_data)
print(daily_dataframe_2022)



Coordinates 52.5483283996582°N 13.407821655273438°E
Elevation 38.0 m asl
Timezone b'Europe/Berlin' b'CEST'
Timezone difference to GMT+0 7200 s
                         date  temperature_2m_max  temperature_2m_min  \
0   2021-12-31 22:00:00+00:00           12.108500            9.158501   
1   2022-01-01 22:00:00+00:00           11.708500            7.708500   
2   2022-01-02 22:00:00+00:00           10.108500            6.358500   
3   2022-01-03 22:00:00+00:00            7.258500            5.208500   
4   2022-01-04 22:00:00+00:00            4.658500            2.458500   
..                        ...                 ...                 ...   
360 2022-12-26 22:00:00+00:00            5.708500            1.758500   
361 2022-12-27 22:00:00+00:00            7.408500            2.408500   
362 2022-12-28 22:00:00+00:00           12.308500            6.908500   
363 2022-12-29 22:00:00+00:00            8.508500            4.258500   
364 2022-12-30 22:00:00+00:00           16.508501     

In [33]:
daily_dataframe_2022.to_csv('weather_data_2022.csv')

In [34]:
weather_data = pd.concat([daily_dataframe_2020, daily_dataframe_2021, daily_dataframe_2022], axis=0) 
weather_data

,date,temperature_2m_max,temperature_2m_min,temperature_2m_mean,sunrise,sunset,daylight_duration,sunshine_duration,rain_sum,snowfall_sum,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant
0,2019-12-31 22:00:00+00:00,4.058500,-1.0415,1.137667,0,0,27881.642578,22130.132812,0.0,0.0,12.313894,22.680000,255.111343
1,2020-01-01 22:00:00+00:00,5.308500,-3.6915,-0.468583,0,0,27955.460938,22598.525391,0.0,0.0,13.979872,24.480000,225.240738
2,2020-01-02 22:00:00+00:00,6.358500,-1.3915,4.073083,0,0,28035.492188,0.000000,3.2,0.0,22.702845,39.959999,223.363449
3,2020-01-03 22:00:00+00:00,6.308500,2.1585,4.018917,0,0,28121.554688,0.000000,4.5,0.0,32.571545,74.159996,278.523407
4,2020-01-04 22:00:00+00:00,3.758500,-1.8415,1.100167,0,0,28213.480469,22702.185547,0.0,0.0,19.642281,34.919998,269.597626
...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-26 22:00:00+00:00,5.708500,1.7585,3.664750,0,0,27621.601562,14976.836914,0.0,0.0,30.903435,55.079998,254.132889
361,2022-12-27 22:00:00+00:00,7.408500,2.4085,5.352250,0,0,27661.103516,0.000000,2.2,0.0,24.826954,42.480000,212.531967
362,2022-12-28 22:00:00+00:00,12.308500,6.9085,9.035584,0,0,27707.484375,11942.952148,1.4,0.0,28.284639,48.959999,219.097702
363,2022-12-29 22:00:00+00:00,8.508500,4.2585,6.783499,0,0,27761.236328,21435.035156,0.5,0.0,27.193705,45.719997,208.409653


In [39]:
weather_data.to_csv('weather_data_2020-2022.csv')